In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

tqdm.pandas()

In [13]:
url = 'https://search.scielo.org/'

Ano de publicação = (year_cluster:(*))

Autor = (au:(*))

Título = (ti:(*))

Resumo = (ab:(*))



# Funções

## Requisição

In [14]:
#Função para fazer a requisição e obter os dados
def buscar_artigos(consulta, n_artigos_pg = 15, pg = 1, sorting=None, anos = None):
    """
    Função para buscar artigos no site da SciELO.
    Parâmetros:
    - consulta: string          # termo ou termos de pesquisa
    - n_artigos_pg: int         # número de artigos por página
    - pg: int                   # número da página
    - sorting: string           # critério de ordenação
    - anos: int ou list of int  # anos de publicação dos artigos
    """

    # Tratamento do parâmetro sorting
    if sorting == 'Mais novos':
        sorting = 'YEAR_DESC'
    elif sorting == 'Mais antigos':
        sorting = 'YEAR_ASC'
    elif sorting == 'Mais relevantes':
        sorting = 'RELEVANCE'
    elif sorting == 'Mais citados':
        sorting = 'CITED_DESC'
    elif sorting == 'Mais acessados':
        sorting = 'ACCESS_DESC'
    elif sorting == None:
        sorting = ''


    params = {
        "q": consulta,
        "lang": "pt",
        "count": n_artigos_pg,
        "output": "site",
        "sort": sorting,
        "format": "summary",
        "page": pg
    }
    if anos:
        if isinstance(anos, list):
            params.update({f"filter[year_cluster][]": ano} for ano in anos)
        else:
            params[f"filter[year_cluster][]"] = anos


    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Erro ao acessar a página: {response.status_code}")
        return None

## Extrair dados da busca

In [15]:
def extrair_artigos(html):
    """
    Função para extrair os dados dos artigos da página de busca.
    Parâmetros:
    - html: string
    """
    # Parse do HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Encontrar os elementos (divs) que contêm os dados dos artigos
    artigos = soup.find_all('div', class_='item')

    resultados = []
    for artigo in artigos:
        try:
            # Encontrar e extrair o título
            titulo_elem = artigo.find('strong', class_='title')
            titulo = titulo_elem.text.strip() if titulo_elem else 'Título não encontrado'

            # Encontrar e extrair o link
            link_elem = artigo.find('a')
            link = link_elem['href'] if link_elem else 'Link não encontrado'

            # Encontrar e extrair o DOI
            doi_elem = artigo.find('div', class_='line metadata')
            if doi_elem:
                doi_link_elem = doi_elem.find('a')
                doi = doi_link_elem['href'] if doi_link_elem else 'DOI não encontrado'
            else:
                doi = 'DOI não encontrado'

            # Encontrar e extrair o resumo
            abstract_elem = artigo.find('div', id=lambda x: x and x.endswith('_pt'))
            abstract = abstract_elem.text.strip() if abstract_elem else 'Resumo em português não encontrado'


            # Encontrar e extrair os nomes dos autores
            autores_elementos = artigo.find_all('a', class_='author')
            autores = [autor.text for autor in autores_elementos]
            if not autores:
                autores.append('Autores não encontrados')

            # Adicionar os dados ao DataFrame
            resultados.append({
                'titulo': titulo,
                'autores': autores,
                'doi': doi,
                'link': link,
                'resumo': abstract
            })
            df = pd.DataFrame(resultados)

            # Remover duplicatas com base no título, mantendo apenas a primeira ocorrência
            df = df.drop_duplicates(subset='titulo', keep='first')
            df = df.reset_index(drop=True)

        except AttributeError as e:
            print(f"Erro ao extrair artigo: {e}")
    return df

# Teste

In [16]:

consulta = '"políticas públicas" AND "saúde"'

n_artigos_pg = 5
    
anos = (2012,2010)

html = buscar_artigos(consulta, n_artigos_pg, sorting = 'Mais antigos')

artigos = extrair_artigos(html)

In [17]:
artigos

,titulo,autores,doi,link,resumo
0,"Administração por convênios, um instrumento ge...","[Sá, Evelin Naked de Castro, Rabello, Maria Ce...",DOI não encontrado,http://www.scielo.br/scielo.php?script=sci_art...,Foi realizada análise dos 1.141 convênios vige...
1,Transição e movimentos sociais contribuição ao...,"[Costa, Nilson do Rosário]",http://analytics.scielo.org/w/bibliometrics/do...,http://www.scielo.br/scielo.php?script=sci_art...,Estudo dedicado a analisar o desenvolvimento d...
2,Abordagem antropológica para avaliação de polí...,"[Minayo, Maria Cecília de Souza]",http://analytics.scielo.org/w/bibliometrics/do...,http://www.scielo.br/scielo.php?script=sci_art...,Apresenta-se uma abordgem antropológica relati...


In [37]:
artigos["link"][0]

'http://www.scielo.br/scielo.php?script=sci_arttext&pid=S0034-89101988000200011&lang=pt'

## Extração do Artigo

Passar por cada link na tabela e raspar o conteúdo de sua página.

.articleSection

In [7]:
import requests
from bs4 import BeautifulSoup
import time

def extrair_secao_artigo(link):
    """
    Extrai o texto da seção do artigo de uma página do SciELO.
    Parâmetros:
    - link: string
    """

    try:
        response = requests.get(link)
        time.sleep(2)
    
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            conteudo_artigo = soup.find('div', class_='articleSection', attrs={'data-anchor': 'Text'})

            if conteudo_artigo:
                heading_tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
                special_tags = heading_tags + ['p']
                headings = []

                for tag in conteudo_artigo.find_all(special_tags):
                    if tag.name in heading_tags:
                        headings.append(tag)
                    elif tag.name == 'p' and tag.find('b'):
                        headings.append(tag.find('b'))
                
                sections = []
                current_section = None

                for heading in headings:
                    if current_section:
                        sections.append(current_section)
                
                    current_section = {
                        'heading': heading.get_text(),
                        'content': ''
                    }

                    # Use find_next_siblings within the parent context
                    for sibling in heading.find_parent().find_next_siblings():
                        if sibling.name in special_tags and (sibling.name in heading_tags or (sibling.name == 'p' and sibling.find('b'))):
                            break
                        if sibling.name == 'p':
                            current_section['content'] += sibling.get_text(separator=' ', strip=True) + ' '
                    
                if current_section is not None:
                    sections.append(current_section)
                    
                filtered_sections = [section for section in sections if section['heading'] not in ['RESUMO', 'ABSTRACT'] and len(section['content'].strip()) > 0]

                return filtered_sections
        else:
            return 'Erro ao acessar a página'
    
    except requests.exceptions.RequestException as e:
        return f'Erro de requisição: {str(e)}'
    
    except Exception as e:
        return f'Erro: {str(e)}'

In [8]:
def extrair_texto_completo(secoes):
    """
    Extrai o texto completo dos artigos.
    Parâmetros:
    - df: DataFrame
    """

    # Adicionar uma coluna ao DataFrame para armazenar o texto completo
    texto_completo = ''
    for secao in secoes:
        texto_completo += f"{secao['heading']}\n{secao['content']}\n\n"

    return texto_completo

In [9]:
artigos['secoes_dict'] = artigos['link'].progress_apply(extrair_secao_artigo)
artigos['texto_completo'] = artigos['secoes_dict'].progress_apply(extrair_texto_completo)

100%|██████████| 3/3 [00:00<00:00, 5572.59it/s]


In [11]:
artigos.to_excel('artigos_scielo.xlsx', index=False)

In [41]:
artigos

,titulo,autores,doi,link,resumo,secoes_dict,texto_completo
0,"Administração por convênios, um instrumento ge...","[Sá, Evelin Naked de Castro, Rabello, Maria Ce...",DOI não encontrado,http://www.scielo.br/scielo.php?script=sci_art...,Foi realizada análise dos 1.141 convênios vige...,[{'heading': 'Evelin Naked de Castro SáI; Mari...,Evelin Naked de Castro SáI; Maria Celina Guima...
1,Transição e movimentos sociais contribuição ao...,"[Costa, Nilson do Rosário]",http://analytics.scielo.org/w/bibliometrics/do...,http://www.scielo.br/scielo.php?script=sci_art...,Estudo dedicado a analisar o desenvolvimento d...,"[{'heading': 'I - INTRODUÇÃO', 'content': ' À ...","I - INTRODUÇÃO\n À medida que, nos últimos ano..."
2,Abordagem antropológica para avaliação de polí...,"[Minayo, Maria Cecília de Souza]",http://analytics.scielo.org/w/bibliometrics/do...,http://www.scielo.br/scielo.php?script=sci_art...,Apresenta-se uma abordgem antropológica relati...,"[{'heading': 'Maria Cecília de Souza Minayo', ...",Maria Cecília de Souza Minayo\n Departamento d...


In [44]:
artigos["secoes_dict"][2]

[{'heading': 'Maria Cecília de Souza Minayo',
  'content': ' Departamento de Ciências Sociais da Escola Nacional de Saúde Pública da Fundação Oswaldo Cruz \x97 Rio de Janeiro, RJ \x97 Brasil  '},
 {'heading': 'Introdução',
  'content': ' O presente artigo traz o debate no fórum, tanto da antropologia como da medicina social a utilização da abordagem antropológica para avaliação de programas de saúde. O principal ponto a ser sublinhado é que, na proposta em questão, a Antropologia sai de seu campo tradicional de pesquisa básica para se colocar, de forma interdisciplinar, com a medicina social e sob o enfoque da avaliação de políticas sociais. Portanto, ela propõe aqui o caráter prioritário de geração de teoria (ainda que o faça concomitantemente). Coloca-se a serviço de um processo social mais abrangente, cujo enfoque é dado pelo programa a ser avaliado, e ao qual serve com seus instrumentos peculiares de apreensão da realidade.  Objetiva-se no presente trabalho, mostrar que o encontro 

In [14]:
artigos["link"][0]

'http://www.scielosp.org/scielo.php?script=sci_arttext&pid=S0103-11042024000200203&lang=pt'

- iterar pelas páginas ?
- Pré processamento?
 - A partir da palavra Introdução ou INTRODUÇÂO remover todo texto anterior?


In [ ]:
import re

def remover_palavras_anteriores(texto, palavra_chave):
    # Encontrar a posição da palavra-chave ignorando maiúsculas/minúsculas
    match = re.search(r'\b' + re.escape(palavra_chave) + r'\b', texto, re.IGNORECASE)

    if match:
        # Capturar o índice da palavra-chave encontrada
        indice = match.start()

        # Retornar a substring a partir do índice da palavra-chave
        texto_limpo = texto[indice:]
        return texto_limpo
    else:
        # Se a palavra-chave não for encontrada, retorna o texto original
        return texto


In [ ]:

texto_original = artigos['secao_artigo'][0]
palavra_chave = "introdução"

texto_limpo = remover_palavras_anteriores(texto_original, palavra_chave)
texto_limpo

'INTRODUÇÃO INTRODUÇÃO  O convênio é uma forma de colaboração, pela qual as instituições podem atingir parte de seus objetivos, sem a necessidade de aumentar sua capacidade instalada e pessoal permanente.  Convênio, vocábulo originário do latim "cum + venire", significa ação conjunta de aproximação, quase sinônimo de acordo \x97 que sugere mais a idéia de superação de um conflito ou a preocupação de preveni-lo \x97, e visa a construir as bases de uma colaboração mútua.  A administração por convênios como instrumento de execução de políticas públicas favorece, na função de administração, a participação periférica nos níveis centrais do poder público no sentido de abordar os assuntos sociais de forma técnica e organizada.  A Secretaria de Estado da Saúde de São Paulo vem adotando largamente o convênio como instrumento de execução de suas próprias finalidades, ou outras que lhe são delegadas, contando, no primeiro semestre de 1987, com cerca de 1.141 convênios, abrangendo um largo campo d

In [58]:
artigos

,titulo,autores,doi,link,resumo,secoes_dict,texto_completo
0,"Administração por convênios, um instrumento ge...","[Sá, Evelin Naked de Castro, Rabello, Maria Ce...",DOI não encontrado,http://www.scielo.br/scielo.php?script=sci_art...,Foi realizada análise dos 1.141 convênios vige...,[{'heading': 'Evelin Naked de Castro SáI; Mari...,Evelin Naked de Castro SáI; Maria Celina Guima...
1,Transição e movimentos sociais contribuição ao...,"[Costa, Nilson do Rosário]",http://analytics.scielo.org/w/bibliometrics/do...,http://www.scielo.br/scielo.php?script=sci_art...,Estudo dedicado a analisar o desenvolvimento d...,"[{'heading': 'I - INTRODUÇÃO', 'content': ' À ...","I - INTRODUÇÃO\n À medida que, nos últimos ano..."
2,Abordagem antropológica para avaliação de polí...,"[Minayo, Maria Cecília de Souza]",http://analytics.scielo.org/w/bibliometrics/do...,http://www.scielo.br/scielo.php?script=sci_art...,Apresenta-se uma abordgem antropológica relati...,"[{'heading': 'Maria Cecília de Souza Minayo', ...",Maria Cecília de Souza Minayo\n Departamento d...


- Filtrar tudo que vem antes de "Introdução" ou "INTRODUÇÃO" e depois de "Referências" ou "REFERÊNCIAS"

In [90]:
from transformers import pipeline
import torch

In [91]:
model_path = 'facebook/bart-large-cnn'
summarization_model = pipeline('summarization', model=model_path, device=-1)

In [97]:
def resumir_artigo(secoes, summarization_model):
    for secao in secoes:
        content = secao['content']
        
        if len(content) > 2000:
            chunks = [content[i:i+2000] for i in range(0, len(content), 2000)]
            summary_length = max(len(chunks) // 3, 1)
            summaries = [summarization_model(chunk, max_length=summary_length, min_length = 10)[0]['summary_text'] for chunk in chunks]
            secao['resumo'] = " ".join(summaries)
        else:
            summary_length = max(len(content) // 3, 1)
            secao['resumo'] = summarization_model(content, max_length=summary_length, min_length = 10)[0]['summary_text']
    
    resumo_completo = " ".join(secao['resumo'] for secao in secoes)

    return resumo_completo

In [98]:
artigos['resumo_llm'] = artigos['secoes_dict'].progress_apply(resumir_artigo)

100%|██████████| 3/3 [08:22<00:00, 167.46s/it]
